
Prediction of the melt pool size using CNN
==============================================
**Author**: `Zaixi Shang, Yaxin Cui and Chengyang Wu`

In [33]:
%matplotlib inline

In [34]:
# Include packages
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from skimage import io, transform

import copy
import os.path
import glob
from torch.utils.data import Dataset, DataLoader
import pandas as pd
plt.ion()   # interactive mode
from matplotlib.pyplot import imshow


Load Data
---------
Load in pre-processed images and labels in training set and validation set. The pre-processed image contains the information of command line, and the lables are the melt pool sizes.


In [35]:
# Writing a custermized dataset for my data
class CommendDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, root_dir , label_dir  , phase, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        #self.num_image = glob.glob(os.path.join(root_dir,'*.png'))
        if phase == 'train':
            self.num_image = 80000
        else:
            self.num_image = 20000
        self.df_label = pd.read_csv(os.path.join(label_dir, phase + '.csv'))
        self.phase = phase

    def __len__(self):
        return self.num_image

    def __getitem__(self, idx):
        lab = self.df_label.iloc[idx]['pixel']
        # if self.phase == 'val':
        #     idx = idx+1999
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                str(idx)+'.png')
        image = io.imread(img_name).transpose((2, 0, 1))

        if self.transform:
            image = self.transform(image)
        
        return image,lab

In [36]:
# Data augmentation and normalization for training
data_transforms = transforms.Compose([
        transforms.CenterCrop(40)
    ])
data_dir = './layers/'
label_dir = './layers/'
image_datasets = {x: CommendDataset(root_dir = os.path.join(data_dir, x),label_dir =label_dir,transform=None ,phase = x) for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=100,
                                             shuffle=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Training the model
------------------


### Define regression model 

In [37]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 9999999
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            epoch_loss = 0
            epoch_acc = 0
            # Iterate over data.
            epo_count = 0
            for inputs, labels in dataloaders[phase]:
                epo_count += 1
                inputs = inputs.to(device, dtype=torch.float32)
                labels = labels.to(device, dtype=torch.float32)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                epoch_loss += loss * inputs.size(0)
                if (epo_count % 10) == 0:
                    num_sample = inputs.size(0)*epo_count
                    cur_RMSE = epoch_loss/num_sample
                    cur_RMSE = cur_RMSE.sqrt()
                    print('{} number of samples: {} RMSE: {:.4f}'.format(phase, num_sample, cur_RMSE))
                    if phase == 'val' and cur_RMSE < best_acc:
                        best_acc = cur_RMSE
                        best_model_wts = copy.deepcopy(model.state_dict()) 
                        print('Best performace, saving model...')
                        torch.save(best_model_wts,'./state/model.p')
                # statistics
                # running_loss += loss.item() * inputs.size(0)
                # running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()
            epoch_loss = epoch_loss/dataset_sizes[phase]
            epoch_acc = epoch_loss.sqrt()
            print('{} MSE: {:.4f} RMSE: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            # deep copy the model
            if phase == 'val' and epoch_acc < best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

### Define the CNN structure

In [39]:
class Regression(nn.Module):
    def __init__(self):
        super(Regression, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 128, 128]
        # our dimension [3,40,40]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 60, 60]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 30, 30]
            nn.ZeroPad2d(1), # [64,32,32]
    

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 32, 32]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 16, 16]
    

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 16, 16]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 8, 8]
 

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
            
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 4, 4]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 2, 2]
        )
        self.cnn = nn.DataParallel(self.cnn)
        self.fc = nn.Sequential(
            nn.Linear(512*2*2, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

Running the model
----------------------

### Test with different types of CNN

In [49]:
num_ca = 1
model_use = 'resnext50_32x4d'

if model_use == 'resnet18': 
    model_ft = models.resnet18(pretrained=False)
    num_ftrs = model_ft.fc.in_features

    model_ft.fc = nn.Linear(num_ftrs,num_ca)
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

elif model_use == 'alexnet': 
    model_ft = models.alexnet(pretrained=True)
    num_ftrs = model_ft.classifier[-1].in_features
    model_ft.classifier[-1] = nn.Linear(num_ftrs,num_ca)
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
    
elif model_use == 'diy':
    model_ft = Regression()
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=1e-5, momentum=0.9)
    
elif model_use == 'resnext50_32x4d': 
    model_ft = models.resnext50_32x4d(pretrained=False)
    num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
    model_ft.fc = nn.Linear(num_ftrs,num_ca)
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=5e-8, momentum=0.9)
    

model_ft = model_ft.to(device)


criterion = nn.MSELoss() 

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=4, gamma=0.1)

### Run the Model

In [50]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=20)

Epoch 0/19
----------
train number of samples: 1000 RMSE: 743.5963
train number of samples: 2000 RMSE: 736.3068
train number of samples: 3000 RMSE: 728.6257
train number of samples: 4000 RMSE: 722.0153
train number of samples: 5000 RMSE: 715.8124
train number of samples: 6000 RMSE: 709.4684
train number of samples: 7000 RMSE: 702.4059
train number of samples: 8000 RMSE: 695.1551
train number of samples: 9000 RMSE: 689.3566
train number of samples: 10000 RMSE: 682.2864
train number of samples: 11000 RMSE: 675.7312
train number of samples: 12000 RMSE: 669.0421
train number of samples: 13000 RMSE: 661.7626
train number of samples: 14000 RMSE: 654.9471
train number of samples: 15000 RMSE: 648.5483
train number of samples: 16000 RMSE: 641.9645
train number of samples: 17000 RMSE: 635.3396
train number of samples: 18000 RMSE: 628.7816
train number of samples: 19000 RMSE: 622.5425
train number of samples: 20000 RMSE: 615.8357
train number of samples: 21000 RMSE: 609.0427
train number of sampl

train number of samples: 78000 RMSE: 153.2327
train number of samples: 79000 RMSE: 153.2504
train number of samples: 80000 RMSE: 153.2098
train MSE: 23473.2402 RMSE: 153.2098
val number of samples: 1000 RMSE: 153.9688
val number of samples: 2000 RMSE: 151.8927
val number of samples: 3000 RMSE: 153.1555
val number of samples: 4000 RMSE: 154.1655
val number of samples: 5000 RMSE: 154.6669
val number of samples: 6000 RMSE: 154.3425
val number of samples: 7000 RMSE: 152.9734
val number of samples: 8000 RMSE: 152.4029
val number of samples: 9000 RMSE: 153.7312
val number of samples: 10000 RMSE: 154.0126
val number of samples: 11000 RMSE: 154.1940
val number of samples: 12000 RMSE: 153.7454
val number of samples: 13000 RMSE: 154.0330
val number of samples: 14000 RMSE: 154.0575
val number of samples: 15000 RMSE: 153.2526
val number of samples: 16000 RMSE: 153.4170
val number of samples: 17000 RMSE: 153.4647
val number of samples: 18000 RMSE: 152.4048
val number of samples: 19000 RMSE: 152.542

train number of samples: 54000 RMSE: 153.4850
train number of samples: 55000 RMSE: 153.5186
train number of samples: 56000 RMSE: 153.6112
train number of samples: 57000 RMSE: 153.5201
train number of samples: 58000 RMSE: 153.6108
train number of samples: 59000 RMSE: 153.4621
train number of samples: 60000 RMSE: 153.2608
train number of samples: 61000 RMSE: 153.3197
train number of samples: 62000 RMSE: 153.2800
train number of samples: 63000 RMSE: 153.1747
train number of samples: 64000 RMSE: 153.0226
train number of samples: 65000 RMSE: 152.8887
train number of samples: 66000 RMSE: 152.9366
train number of samples: 67000 RMSE: 153.0054
train number of samples: 68000 RMSE: 152.8927
train number of samples: 69000 RMSE: 153.0170
train number of samples: 70000 RMSE: 153.1376
train number of samples: 71000 RMSE: 153.1057
train number of samples: 72000 RMSE: 153.0052
train number of samples: 73000 RMSE: 153.0915
train number of samples: 74000 RMSE: 153.0930
train number of samples: 75000 RMS

train number of samples: 31000 RMSE: 151.6107
train number of samples: 32000 RMSE: 151.6251
train number of samples: 33000 RMSE: 151.7278
train number of samples: 34000 RMSE: 151.5807
train number of samples: 35000 RMSE: 151.5028
train number of samples: 36000 RMSE: 151.6682
train number of samples: 37000 RMSE: 151.6131
train number of samples: 38000 RMSE: 151.8483
train number of samples: 39000 RMSE: 152.1068
train number of samples: 40000 RMSE: 152.0471
train number of samples: 41000 RMSE: 152.0542
train number of samples: 42000 RMSE: 152.1422
train number of samples: 43000 RMSE: 152.3123
train number of samples: 44000 RMSE: 152.0896
train number of samples: 45000 RMSE: 152.1939
train number of samples: 46000 RMSE: 152.3573
train number of samples: 47000 RMSE: 152.4111
train number of samples: 48000 RMSE: 152.4454
train number of samples: 49000 RMSE: 152.4382
train number of samples: 50000 RMSE: 152.3728
train number of samples: 51000 RMSE: 152.4731
train number of samples: 52000 RMS

train number of samples: 7000 RMSE: 149.8245
train number of samples: 8000 RMSE: 149.7357
train number of samples: 9000 RMSE: 149.1241
train number of samples: 10000 RMSE: 149.6140
train number of samples: 11000 RMSE: 150.4336
train number of samples: 12000 RMSE: 150.9380
train number of samples: 13000 RMSE: 151.2086
train number of samples: 14000 RMSE: 150.9680
train number of samples: 15000 RMSE: 150.3103
train number of samples: 16000 RMSE: 150.7884
train number of samples: 17000 RMSE: 150.9578
train number of samples: 18000 RMSE: 151.1557
train number of samples: 19000 RMSE: 150.4173
train number of samples: 20000 RMSE: 150.8727
train number of samples: 21000 RMSE: 151.1033
train number of samples: 22000 RMSE: 151.0432
train number of samples: 23000 RMSE: 151.1326
train number of samples: 24000 RMSE: 151.4474
train number of samples: 25000 RMSE: 151.1479
train number of samples: 26000 RMSE: 150.7541
train number of samples: 27000 RMSE: 150.7809
train number of samples: 28000 RMSE: 

...

train number of samples: 39000 RMSE: 152.5477
train number of samples: 40000 RMSE: 152.6207
train number of samples: 41000 RMSE: 152.6724
train number of samples: 42000 RMSE: 152.8038
train number of samples: 43000 RMSE: 152.9491
train number of samples: 44000 RMSE: 153.0054
train number of samples: 45000 RMSE: 152.7420
train number of samples: 46000 RMSE: 152.7125
train number of samples: 47000 RMSE: 152.5951
train number of samples: 48000 RMSE: 152.6691
train number of samples: 49000 RMSE: 152.6331
train number of samples: 50000 RMSE: 152.7847
train number of samples: 51000 RMSE: 152.7670
train number of samples: 52000 RMSE: 152.6146
train number of samples: 53000 RMSE: 152.5832
train number of samples: 54000 RMSE: 152.4602
train number of samples: 55000 RMSE: 152.4783
train number of samples: 56000 RMSE: 152.2366
train number of samples: 57000 RMSE: 152.1085
train number of samples: 58000 RMSE: 152.1883
train number of samples: 59000 RMSE: 152.5173
train number of samples: 60000 RMS

train number of samples: 16000 RMSE: 148.7424
train number of samples: 17000 RMSE: 149.1243
train number of samples: 18000 RMSE: 149.2704
train number of samples: 19000 RMSE: 149.6122
train number of samples: 20000 RMSE: 149.9172
train number of samples: 21000 RMSE: 149.8754
train number of samples: 22000 RMSE: 149.7150
train number of samples: 23000 RMSE: 149.9729
train number of samples: 24000 RMSE: 150.3784
train number of samples: 25000 RMSE: 150.2715
train number of samples: 26000 RMSE: 150.7123
train number of samples: 27000 RMSE: 150.9129
train number of samples: 28000 RMSE: 150.8506
train number of samples: 29000 RMSE: 150.9858
train number of samples: 30000 RMSE: 151.4985
train number of samples: 31000 RMSE: 151.6725
train number of samples: 32000 RMSE: 151.4974
train number of samples: 33000 RMSE: 151.2641
train number of samples: 34000 RMSE: 151.6881
train number of samples: 35000 RMSE: 151.9111
train number of samples: 36000 RMSE: 152.2200
train number of samples: 37000 RMS

val number of samples: 14000 RMSE: 152.7733
val number of samples: 15000 RMSE: 152.9564
val number of samples: 16000 RMSE: 152.5261
val number of samples: 17000 RMSE: 151.9217
val number of samples: 18000 RMSE: 152.4000
val number of samples: 19000 RMSE: 152.3816
val number of samples: 20000 RMSE: 152.5213
val MSE: 23262.7617 RMSE: 152.5213

Epoch 16/19
----------
train number of samples: 1000 RMSE: 150.7849
train number of samples: 2000 RMSE: 151.1540
train number of samples: 3000 RMSE: 151.2541
train number of samples: 4000 RMSE: 151.5315
train number of samples: 5000 RMSE: 150.0582
train number of samples: 6000 RMSE: 150.3445
train number of samples: 7000 RMSE: 152.7153
train number of samples: 8000 RMSE: 153.4878
train number of samples: 9000 RMSE: 153.7790
train number of samples: 10000 RMSE: 154.3106
train number of samples: 11000 RMSE: 154.3338
train number of samples: 12000 RMSE: 153.5720
train number of samples: 13000 RMSE: 152.9346
train number of samples: 14000 RMSE: 153.073

train number of samples: 71000 RMSE: 152.7549
train number of samples: 72000 RMSE: 152.6734
train number of samples: 73000 RMSE: 152.9611
train number of samples: 74000 RMSE: 152.9489
train number of samples: 75000 RMSE: 152.9757
train number of samples: 76000 RMSE: 153.0102
train number of samples: 77000 RMSE: 152.9752
train number of samples: 78000 RMSE: 152.9527
train number of samples: 79000 RMSE: 153.0818
train number of samples: 80000 RMSE: 153.0876
train MSE: 23435.8223 RMSE: 153.0876
val number of samples: 1000 RMSE: 157.0324
val number of samples: 2000 RMSE: 164.7168
val number of samples: 3000 RMSE: 160.4362
val number of samples: 4000 RMSE: 157.8567
val number of samples: 5000 RMSE: 156.8989
val number of samples: 6000 RMSE: 156.4377
val number of samples: 7000 RMSE: 157.2569
val number of samples: 8000 RMSE: 156.2041
val number of samples: 9000 RMSE: 156.4714
val number of samples: 10000 RMSE: 156.1692
val number of samples: 11000 RMSE: 155.2541
val number of samples: 12000

train number of samples: 48000 RMSE: 151.5238
train number of samples: 49000 RMSE: 151.5816
train number of samples: 50000 RMSE: 151.7108
train number of samples: 51000 RMSE: 151.8802
train number of samples: 52000 RMSE: 151.8927
train number of samples: 53000 RMSE: 151.7921
train number of samples: 54000 RMSE: 152.2169
train number of samples: 55000 RMSE: 152.4173
train number of samples: 56000 RMSE: 152.1525
train number of samples: 57000 RMSE: 152.1190
train number of samples: 58000 RMSE: 152.1882
train number of samples: 59000 RMSE: 152.2398
train number of samples: 60000 RMSE: 152.2143
train number of samples: 61000 RMSE: 152.1721
train number of samples: 62000 RMSE: 152.2487
train number of samples: 63000 RMSE: 152.3572
train number of samples: 64000 RMSE: 152.5801
train number of samples: 65000 RMSE: 152.6853
train number of samples: 66000 RMSE: 152.8094
train number of samples: 67000 RMSE: 153.0004
train number of samples: 68000 RMSE: 152.8968
train number of samples: 69000 RMS